In [1]:
from pathlib import Path
thispath = Path('__file__').resolve()
base_path = thispath.parent.parent
import sys; sys.path.insert(0, str(base_path))
import json
sys.path.append('../utils/')
import time

In [2]:
import SimpleITK as sitk
import pandas as pd
from tqdm.auto import tqdm
import time
import numpy as np
from metrics import get_landmarks_array_from_txt_file, target_registration_error
from elastic_helpers import elastix_wrapper, modify_field_parameter_map,transformix_wrapper

In [3]:
base_path = Path().resolve().parent
params_path = base_path / 'parameters'
output_path = base_path /'elastix'
results_path = base_path / 'experiments' / 'elastik' / 'results.json'


In [4]:
dataset = pd.read_csv('../result_data/dir_lab_copd.csv')

In [5]:
def registration(dataset,param_maps_to_use,experiment_name,masks=False):
    results = {}

    for i in tqdm(range(len(dataset))):
        sample = dataset.iloc[i]
        sample_data = sample['case']
        sample_mean = sample['disp_mean']
        sample_std = sample['disp_std']

        pm = param_maps_to_use[-1]
        data_path = base_path / 'result_data' / sample_data
        output_pm_path = output_path / pm.rstrip('.txt') / experiment_name
        output_pm_path.mkdir(exist_ok=True, parents=True)

        parameters_filename = params_path
        result_path = output_path / sample['case'] / experiment_name
        result_path.mkdir(exist_ok=True, parents=True)
        
        param_maps_to_use_ = [str(params_path / p) for p in param_maps_to_use]
        name = sample['case']
        i_temp_path = data_path / f'{sample_data}_iBHCT.nii.gz'
        e_temp_path = data_path / f'{sample_data}_eBHCT.nii.gz'
        i_lungs_mask_temp_path = data_path / f'{sample_data}_iBHCT_masks.nii.gz'
        e_lungs_mask_temp_path = data_path / f'{sample_data}_eBHCT_masks.nii.gz'
        
        print('Registration of image...')
        start = time.time()
        
        if masks is False:
            transform_map_path = elastix_wrapper(
                i_temp_path, e_temp_path, result_path.parent, param_maps_to_use_,
                verbose=False, keep_just_useful_files=False
            )
        else:
            transform_map_path = elastix_wrapper(
                i_temp_path, e_temp_path, result_path.parent, param_maps_to_use_,
                i_lungs_mask_temp_path, e_lungs_mask_temp_path, verbose=False,
                keep_just_useful_files=False
            )
        
        
        reg_time = time.time() - start
        name = f'{sample_data}_300_iBH_xyz_r1.txt'
        lm_points_filepath = data_path / name
        
        field_value_pairs = [
            ('ResultImageFormat', 'nii.gz'),
            ('ResultImagePixelType', "float"),
            ('FinalBSplineInterpolationorder', '0')
        ]

        modify_field_parameter_map(field_value_pairs, transform_map_path)
        
        # Transform landmarks
        print('Transforming points...')
        lm_out_filepath = result_path.parent / f'r_{sample_data}'
        transformix_wrapper(
            lm_points_filepath, lm_out_filepath, transform_map_path,
            points=True)
                
        landmarks = get_landmarks_array_from_txt_file(lm_out_filepath / name)
        e_landmark_pts = pd.read_csv(sample['e_landmark_pts'], header=None, index_col=None).values
        m, s = target_registration_error(landmarks, e_landmark_pts, [sample['space_x'],sample['space_y'],sample['space_z']])
        print(f'{"-"*10} {sample_data} {"-"*10}')
        print(f'Provided displacement: {sample_mean} | {sample_std} ')
        print(f'Computed displacement: {m} | {s}')
        
        results[sample['case']] = {
            'mean_tre': m,
            'std_tre': s,
            'time': reg_time,
            'maks_on' : masks
        }
    
    return results

## Experiments without a mask

In [6]:
experiment_results = {}
experiment_name = 'Baseline'
param_maps_to_use = [
        'Par0054_sstvd.txt'
]

In [7]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 21.03 | 10.12
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 18.76 | 6.0
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 7.59 | 4.84
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 24.55 | 9.36


In [8]:
experiment_name = 'Par0054_resolutions_6'
param_maps_to_use = [
        'Par0054_resolutions_6.txt'
]

In [9]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 8.7 | 6.77
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.9 | 6.14
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.97 | 3.83
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 11.01 | 4.8


In [17]:
experiment_name = 'Par0054_resolutions_6_600it'
param_maps_to_use = [
        'Par0054_resolutions_6_600it.txt'
]

In [18]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 9.32 | 5.94
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 11.88 | 6.71
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.74 | 4.05
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 11.1 | 5.27


In [19]:
experiment_name = 'Par0054_resolutions_6_200it'
param_maps_to_use = [
        'Par0054_resolutions_6_200it.txt'
]

In [20]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 8.98 | 5.95
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.26 | 6.42
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.79 | 3.95
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 10.64 | 5.11


In [21]:
experiment_name = 'Par0003.bs-R8-fg'
param_maps_to_use = [
        'Par0003.bs-R8-fg.txt'
]

In [22]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.52 | 5.16
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.37 | 7.35
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.55 | 3.62
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 9.02 | 4.65


In [25]:
experiment_name = 'Par0003.bs-R8-fg_bspline'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.11 | 4.43
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.33 | 7.07
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.07 | 3.16
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.8 | 4.07


In [26]:
experiment_name = 'Par0003.bs-R8-fg_bspline_SP_a 10000'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_SP_a 10000.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.11 | 4.43
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.33 | 7.07
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.07 | 3.16
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.8 | 4.07


In [27]:
experiment_name = 'Par0003.bs-R8-fg_bspline_SP_A 50'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_SP_A 50.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.02 | 4.45
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.13 | 6.92
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.1 | 3.08
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.66 | 4.01


In [29]:
experiment_name = 'Par0003.bs-R8-fg_bspline_MAXITER_3000'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_MAXITER_3000.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 6.99 | 4.6
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.11 | 7.18
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.02 | 3.25
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 8.13 | 4.32


In [30]:
experiment_name = 'Par0003.bs-R8-fg_BSPLINE_12UNITS'
param_maps_to_use = [
        'Par0003.bs-R8-fg_BSPLINE_12UNITS.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.17 | 4.6
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.4 | 7.01
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.19 | 3.27
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 8.04 | 4.17


In [28]:
experiment_name = 'Par0003.bs-R8-fg_affine'
param_maps_to_use = [
        'Par0003.bs-R8-fg_affine.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.17 | 4.6
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.4 | 7.01
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.19 | 3.27
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 8.04 | 4.17


In [31]:
experiment_name = 'Par0003.bs-6R_modified'
param_maps_to_use = [
        'Par0003.bs-6R_modified.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.13 | 4.45
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.39 | 7.09
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.04 | 3.17
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.8 | 4.11


In [32]:
experiment_name = 'Par0003.affine'
param_maps_to_use = [
        'Par0003.affine.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 25.91 | 10.99
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 24.61 | 5.38
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.9 | 3.33
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 21.62 | 9.58


In [35]:
experiment_name = 'best_untilnow'
param_maps_to_use = [
        'best_untilnow.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.17 | 4.6
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.4 | 7.01
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.19 | 3.27
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 8.04 | 4.17


In [37]:
experiment_name = 'Par0003.bs-R8-fg_NormalizedMI'
param_maps_to_use = [
        'Par0003.bs-R8-fg_NormalizedMI.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 4.36 | 4.0
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 5.84 | 6.07
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 2.93 | 2.69
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 5.58 | 5.28


## Experiments with masks

In [40]:
results_path = base_path / 'experiments' / 'elastik' / 'results_masks.json'
experiment_results = {}

In [41]:
experiment_name = 'Baseline'
param_maps_to_use = [
        'Par0054_sstvd.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 25.6 | 11.69
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 21.18 | 6.51
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 6.71 | 6.61
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 27.99 | 12.22


In [46]:
experiment_name = 'Par0003.bs-R8-fg'
param_maps_to_use = [
        'Par0003.bs-R8-fg.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.38 | 1.75
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 3.69 | 6.94
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.09 | 0.97
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.47 | 1.2


In [47]:
experiment_name = 'Par0003.bs-R8-fg_bspline_SP_a 10000'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_SP_a 10000.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 7.11 | 4.43
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 12.33 | 7.07
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.07 | 3.16
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.8 | 4.07


In [48]:
experiment_name = 'Par0003.bs-R8-fg_bspline_SP_A 50'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_SP_A 50.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.3 | 1.28
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.89 | 4.03
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.13 | 0.98
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.48 | 1.16


In [49]:
experiment_name = 'Par0003.bs-R8-fg_bspline_MAXITER_3000'
param_maps_to_use = [
        'Par0003.bs-R8-fg_bspline_MAXITER_3000.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.25 | 1.24
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.73 | 4.12
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.12 | 0.95
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.4 | 1.12


In [50]:
experiment_name = 'Par0003.bs-R8-fg_BSPLINE_12UNITS'
param_maps_to_use = [
        'Par0003.bs-R8-fg_BSPLINE_12UNITS.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.32 | 1.44
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.86 | 4.04
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.15 | 0.99
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.46 | 1.12


In [51]:
experiment_name = 'Par0003.bs-R8-fg_affine'
param_maps_to_use = [
        'Par0003.bs-R8-fg_affine.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.32 | 1.44
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.86 | 4.04
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.15 | 0.99
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.46 | 1.12


In [52]:
experiment_name = 'Par0003.bs-6R_modified'
param_maps_to_use = [
        'Par0003.bs-6R_modified.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.3 | 1.28
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.79 | 4.04
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.13 | 0.95
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.5 | 1.17


In [53]:
experiment_name = 'Par0003.affine'
param_maps_to_use = [
        'Par0003.affine.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 9.31 | 5.39
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 11.34 | 6.3
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 3.4 | 1.65
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 7.56 | 5.02


In [57]:
experiment_name = 'best_untilnow'
param_maps_to_use = [
        'best_untilnow.txt'
]
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name,masks=True)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 1.32 | 1.44
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 2.86 | 4.04
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 1.15 | 0.99
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 1.46 | 1.12
